## Crassification Wine
　　　From SIGNATE　【練習問題】ワインの品種の予測

※ SIGNATEから元データの再配布が認められていないため、このノートブックで解析している元データ(csvファイル、tsvファイル)についてはお手数ですがSIGNATEの【練習問題】サイトからダウンロードして、このipynbファイルと同じ場所に入れてから実行してください。

In [ ]:
# 基本ライブラリの読み込み
import numpy as np
import pandas as pd

In [ ]:
# train.tsvの読み込み
df_train = pd.read_csv("train.tsv", sep="\t")
df_train.head()

In [ ]:
# インデックスとbot（目的変数）を説明変数から削除してtrainXに代入する
trainX = df_train.iloc[:, 2:]
trainX.head()

In [ ]:
trainX.shape

In [ ]:
trainX.describe()

In [ ]:
trainX.info()

In [ ]:
trainX.isnull().sum()

In [ ]:
# 目的変数をyに代入する
y = df_train.iloc[:,[1]]
display(y.head())

In [ ]:
# test.tsvの読み込み
df_test = pd.read_csv("test.tsv", sep="\t")
df_test.head()

In [ ]:
testX = df_test.iloc[:, 1:]
testX.head()

In [ ]:
testX.shape

In [ ]:
testX.describe()

In [ ]:
testX.info()

In [ ]:
testX.isnull().sum()

In [ ]:
# 交叉検証　Holdout法
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(trainX,
                                                 y,
                                                 test_size=0.30,
                                                 random_state=1)

In [ ]:
# 整形
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

# set pipelines for three different algorithms
pipe_LR = Pipeline([('scl',StandardScaler()),
                       ('est',LogisticRegression(random_state=1))])
pipe_RF = Pipeline([('scl',StandardScaler()),
                        ('est',RandomForestClassifier(random_state=1))])
pipe_GB = Pipeline([('scl',StandardScaler()),
                        ('est',GradientBoostingClassifier(random_state=1))])
pipe_PCARF = Pipeline([('scl',StandardScaler()),
                          ('pca',PCA(random_state=1)),
                          ('est',RandomForestClassifier(random_state=1))])
pipe_PCAGB = Pipeline([('scl',StandardScaler()),
                          ('pca',PCA(random_state=1)),
                          ('est',GradientBoostingClassifier(random_state=1))])
pipe_SVC = Pipeline([('scl',StandardScaler()),
                          ('est',LinearSVC(random_state=1))])
pipe_KNC = Pipeline([('scl',StandardScaler()),
                          ('est',KNeighborsClassifier())])

In [ ]:
# 学習
pipe_LR.fit(X_train, y_train)
pipe_RF.fit(X_train, y_train)
pipe_GB.fit(X_train, y_train)
pipe_PCARF.fit(X_train, y_train)
pipe_PCAGB.fit(X_train, y_train)
pipe_SVC.fit(X_train, y_train)
pipe_KNC.fit(X_train, y_train)

In [19]:
# 評価（accuracyスコアとする）
from sklearn.metrics import accuracy_score
print('pipe_LR_Train:%.3f'% accuracy_score(y_train,
                                          pipe_LR.predict(X_train)))
print('pipe_LR_Test:%.3f' % accuracy_score(y_test,
                                          pipe_LR.predict(X_test)))

print('pipe_RF_Train:%.3f'% accuracy_score(y_train,
                                          pipe_RF.predict(X_train)))
print('pipe_RF_Test:%.3f' % accuracy_score(y_test,
                                          pipe_RF.predict(X_test)))

print('pipe_GB_Train:%.3f'% accuracy_score(y_train,
                                          pipe_GB.predict(X_train)))
print('pipe_GB_Test:%.3f' % accuracy_score(y_test,
                                          pipe_GB.predict(X_test)))

print('pipe_PCARF_Train:%.3f'% accuracy_score(y_train,
                                          pipe_PCARF.predict(X_train)))
print('pipe_PCARF_Test:%.3f' % accuracy_score(y_test,
                                          pipe_PCARF.predict(X_test)))

print('pipe_PCAGB_Train:%.3f'% accuracy_score(y_train,
                                          pipe_PCAGB.predict(X_train)))
print('pipe_PCAGB_Test:%.3f' % accuracy_score(y_test,
                                          pipe_PCAGB.predict(X_test)))

print('pipe_SVC_Train:%.3f'% accuracy_score(y_train,
                                          pipe_SVC.predict(X_train)))
print('pipe_SVC_Test:%.3f' % accuracy_score(y_test,
                                          pipe_SVC.predict(X_test)))

print('pipe_KNC_Train:%.3f'% accuracy_score(y_train,
                                          pipe_KNC.predict(X_train)))
print('pipe_KNC_Test:%.3f' % accuracy_score(y_test,
                                          pipe_KNC.predict(X_test)))

pipe_LR_Train:1.000
pipe_LR_Test:0.963
pipe_RF_Train:1.000
pipe_RF_Test:0.963
pipe_GB_Train:1.000
pipe_GB_Test:0.889
pipe_PCARF_Train:1.000
pipe_PCARF_Test:0.852
pipe_PCAGB_Train:1.000
pipe_PCAGB_Test:0.926
pipe_SVC_Train:1.000
pipe_SVC_Test:0.963
pipe_KNC_Train:0.968
pipe_KNC_Test:0.963


In [ ]:
# パラメータグリッドの設定
from sklearn.model_selection import GridSearchCV

param_grid_RF = {
#                  'pca__n_components':[1,2,3],
                 'est__n_estimators': [5, 10, 20, 30, 50],# 整数を指定．デフォルトの値は10．バギングに用いる決定木の個数を指定．
                 'est__max_features': [5,8,10],
# 整数，小数，文字列または None を指定。デフォルトは None。最適な分割をするために考慮するフィーチャーの数を指定。
# 整数を指定した場合，その個数，小数の場合全フィーチャーに対する割合個，auto を指定した場合，フィーチャー数のルート個
# log2 を指定した場合，log2(フィーチャー数) 個。基本は None を使うべき。
                 'est__random_state': [1],# 乱数のタネの指定。何かしらの整数を指定すべき。
                 'est__n_jobs': [-1],# 複数のCPUコアを使って並列に学習します。-1は最大値。
                 'est__min_samples_split' : [3, 5, 10, 15, 20, 25, 30, 40],
# 整数または小数を指定．デフォルトは None．ノードを分割するために必要な最小サンプルサイズ.
# 整数を指定した場合，その数，小数を指定した場合，全サンプルサイズに対する割合個．
                 'est__max_depth' : [3, 5, 10, 15, 20]}# 整数または None を指定．決定木の深さの最大値を指定．過学習を避けるためにはこれを調節するのが最も重要．

In [ ]:
print('探索空間:%s' %param_grid_RF)
clf = GridSearchCV(estimator=pipe_RF,# 対象の機械学習モデル
                   param_grid=param_grid_RF,# 探索パラメータ辞書
                   scoring='accuracy',
                   cv=3,# クロスバリデーションの分割数
                   return_train_score=False)

In [22]:
clf = clf.fit(X_train, y_train)
print('Best Score:', clf.best_score_)
print('Best Params', clf.best_params_)

Best Score: 0.9516129032258065
Best Params {'est__max_depth': 3, 'est__max_features': 5, 'est__min_samples_split': 10, 'est__n_estimators': 5, 'est__n_jobs': -1, 'est__random_state': 1}


In [ ]:
pred = pipe_RF.predict(testX)
pred

In [ ]:
sample = pd.read_csv("sample_submit.csv",header=None)
sample.head()

In [ ]:
# 提出用csvの正解カラムにpredを代入
sample[1] = pred
sample.head()

In [ ]:
# 提出用データをcsv形式で出力
sample.to_csv("submit0.csv",index=None,header=None)